In [1]:
from func import *

import pickle as p
import glob 
import json
import subprocess
import getpass
import os
# import shutil

In [2]:
# ",".join(os.listdir('/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/lablmaps/KORA'))

In [3]:
dataset='KORA'
volume_txt_file = 'datasets/kora/volumes.txt'
data_dir = "/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti"
label_dir = 'datasets/lablmaps/KORA'

n4_corrected_data_dir = "temp/KORA/n4_corrected"
stictched_data_dir = "temp/KORA/stitched"
stitched_n4_corrected_data_dir = "temp/KORA/stitched_n4_corrected"

processed_path = 'temp/KORA/'
processed_dir = 'temp/KORA/processed'

one_time_n4_optimization = True

In [4]:
# def rename_folder(dir_name,folder_id, prefix='KORA'):
#     create_if_not(f'{dir_name}/{prefix}{folder_id}')
#     os.rename(f'{dir_name}/{folder_id}/**', f'{dir_name}/{prefix}{folder_id}/**')

In [5]:
# for folder_path in os.listdir('/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/lablmaps/KORA'):
#     dir_name = '/'.join(folder_path.split('/')[:-1])
#     folder_id = folder_path.split('/')[-1]
#     rename_folder(dir_name, folder_id)

In [6]:
def load_kora_file_paths(load_from_txt_file=True):
    volumes_to_use = []
    if load_from_txt_file:
        with open(volume_txt_file) as file_handle:
                volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for name in os.listdir(data_dir)]
    
    file_paths = {}
    
    for vol in volumes_to_use:
        if vol == "":
            continue
            
        labelmap_paths = glob.glob(f'{label_dir}/{vol}/**')
#         vol = 'KORA'+vol if dataset=='KORA' else vol
        opp_paths = glob.glob(f'{data_dir}/{vol}/**opp_[0-9]**.nii.gz')
        in_paths = glob.glob(f'{data_dir}/{vol}/**in_[0-9]**.nii.gz')
        f_paths = glob.glob(f'{data_dir}/{vol}/**F_[0-9]**.nii.gz')
        w_paths = glob.glob(f'{data_dir}/{vol}/**W_[0-9]**.nii.gz')
        
        vol_madals_paths = dict(
        OPP=opp_paths,
        IN=in_paths,
        F=f_paths,
        W=w_paths
        )
        file_paths[str(vol)]=dict(
            VOLUME_PATHS=vol_madals_paths,
            LABEL_PATHS=labelmap_paths,
        )
    return file_paths
        

In [7]:
file_paths = load_kora_file_paths()

In [8]:
 print(json.dumps(file_paths,sort_keys=True, indent=4))

{
    "KORA2452879": {
        "LABEL_PATHS": [
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Pancreas.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/Combi",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Liver.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Kidney (left).nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Adrenal gland(right).nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Spleen.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Gallbladder.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Kidney (right).nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe

In [9]:
# Individual RESCALING.
n4_dict = {}
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    print(f'n4 processing part-1 started with {vol}...')
    n4_dict[vol] = []
    vol_parts = [[file, read_ras(file)] for file in file_paths[vol]['VOLUME_PATHS']['IN']]
    for orig_file, in_image in vol_parts:
        n4_dict[vol].append(rescale(in_image, vol, orig_file))

In [10]:
# TODO: EXECUTE 'sudo sh ./n4-anne.sh' from command line for n4 bais field generation of all in imgs.

for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    print(f'applying n4 bias field on opp scans of {vol}...')
    vol_parts = [[read_ras(opp_file), read_ras(n4_counterpart['IN_BIAS']), opp_file] for opp_file, n4_counterpart in zip(file_paths[vol]['VOLUME_PATHS']['OPP'],n4_dict[vol])]
    idx = 0
    file_paths[vol]['N4_1'] = []
    for opp_img, bias_field_img, opp_file in vol_parts:
        file_paths[vol]['N4_1'].append(apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict[vol][idx], vol))
        idx+=1
#         n4_dict[vol].append(rescale(opp_image, vol, n4_dict[vol]))



In [ ]:
# STITCHING VOL PARTS HERE
for vol in file_paths.keys():
    print(f'started with {vol}...')
    create_if_not(f'{n4_corrected_data_dir}/vol/{vol}')
    file_paths[vol]['ONE'] = {}
    for modality_key in file_paths[vol]['VOLUME_PATHS'].keys():
        print(f"processing {modality_key}")
        orig_modal_key = modality_key
        if one_time_n4_optimization:
            vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]
        else:
            if modality_key == 'OPP':
                vol_parts = [read_ras(data_dict['OPP_CORRECTED']) for data_dict in file_paths[vol]['N4_1']]
                modality_key = modality_key+'_n4_corrected'
            else:
                vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]

        ras_stitched = multi_vol_stitching(vol_parts)
        save_volume(ras_stitched, f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched')
        file_paths[vol]['ONE'][f'{orig_modal_key}'] = f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched.nii.gz'

started with KORA2456289...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         235.26948547]
 [  0.           0.          -1.69999695 144.93167114]
 [  0.          -1.70138884   0.         -21.65872765]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          230.26948547]
 [   0.            0.           -1.69999695  144.93167114]
 [   0.           -1.66666663    0.         -256.65872192]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2456289
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         235.26948547]
 [  0.           0.          -1.69999695 144.93167114]
 [  0.          

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          240.02130127]
 [   0.            0.           -1.69999695  119.15551758]
 [   0.           -1.66666663    0.         -215.20669556]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2452879
processing W
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         245.02130127]
 [  0.           0.          -1.69999695 119.15551758]
 [  0.          -1.70138884   0.          19.79330063]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          240.02130127]
 [   0.            0.           -1.69999695  119.15551758]
 [   0.           -1

Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2453229
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         262.03250122]
 [  0.           0.          -1.69999695 152.1910553 ]
 [  0.          -1.70138884   0.           0.52120578]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          257.03250122]
 [   0.            0.           -1.69999695  152.1910553 ]
 [   0.           -1.66666663    0.         -234.47879028]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2453229
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         2

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          240.01109314]
 [   0.            0.           -1.69999695  116.75502014]
 [   0.           -1.66666663    0.         -283.12890625]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2453290
started with KORA2460734...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         242.58334351]
 [  0.           0.          -1.69999695 140.065979  ]
 [  0.          -1.70138884   0.          -9.54386711]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          237.58334351]
 [   0.            0.           -1.69999695  140.

Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2459745
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         249.87677002]
 [  0.           0.          -1.70001221 138.57743835]
 [  0.          -1.70138884   0.         -13.95329475]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          244.87678528]
 [   0.            0.           -1.70001221  138.57743835]
 [   0.           -1.66666663    0.         -248.95329285]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2459745
processing W
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         24

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          235.15220642]
 [   0.            0.           -1.69999695  121.61049652]
 [   0.           -1.66666663    0.         -241.47940063]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2462405
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         240.15220642]
 [  0.           0.          -1.69999695 121.61049652]
 [  0.          -1.70138884   0.          -6.47939968]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          235.15220642]
 [   0.            0.           -1.69999695  121.61049652]
 [   0.           -

In [ ]:
# RESCALING INTENSITIES OF STITCHED VOLUME ABOVE 0
n4_dict = {}
for vol in file_paths.keys():
    print(f'n4 processing part-2 started with {vol}...')
    n4_dict[vol] = {}
    in_stitched_file_path, in_stitched_img = file_paths[vol]['ONE']['IN'], read_ras(file_paths[vol]['ONE']['IN'])
    n4_dict[vol]['N4_2'] = rescale(in_stitched_img, vol, in_stitched_file_path)

In [ ]:
# TODO: EXECUTE 'sudo sh ./n4-anne-2.sh' from command line for n4 bais field generation of all in imgs.
for vol in file_paths.keys():
    print(f'applying n4 bias field on stitched opp scans of {vol}...')

    opp_file = file_paths[vol]['ONE']['OPP']
    n4_counterpart = n4_dict[vol]['N4_2']
    print(opp_file, n4_counterpart['IN_BIAS'])
    opp_img = read_ras(opp_file)
    bias_field_img = read_ras(n4_counterpart['IN_BIAS'])

    file_paths[vol]['N4_2'] = apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict[vol]['N4_2'], vol)

In [ ]:
with open('kora_file_paths.p', 'wb') as handle:
    p.dump(file_paths, handle, protocol=p.HIGHEST_PROTOCOL)

In [ ]:
with open('kora_file_paths.p', 'rb') as handle:
    file_paths = p.load(handle)

In [ ]:
def vol_label_fix(vol, label):
    vol_vol = np.product(vol.shape)
    label_vol = np.product(label.shape)
    
    world_shape = np.max(np.array([list(vol.shape), list(label.shape)]), axis=0)
    print(tuple(world_shape))
    final_label = np.zeros(tuple(world_shape))
    label_affine = label.affine
    vol_affine = vol.affine
    if(vol_vol>label_vol):
#         final_label = np.zeros(vol.shape)
        target_affine = vol_affine
        target_header = vol.header
        target_dim_v = vol.shape
        
#         labelmap2vol = npl.inv(target_affine).dot(label_affine)
#         start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
#         sx, sy,sz = start_inv
#         end_inv = apply_affine(labelmap2vol, target_dim_v).astype(np.int32)
#         final_label = np.zeros(end_inv)
#         ex, ey, ez = end_inv
#         print("seg start inv v: ",start_inv , "segm end inv v:",end_inv)
        sx,sy,sz,ex,ey,ez = np.abs(get_points(label, vol))
        print(sx,sy,sz,ex,ey,ez)
        final_label[0:ex+sx, 0:ey+sy, 0:ez+sz] = label.get_fdata()
        final_label = np.flip(final_label, axis=0)
#         final_label = np.flip(final_label, axis=1)
        final_label = np.flip(final_label, axis=2)
        final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
        volume, label = vol, final_label_img
    else:
#         final_label = np.zeros(label.shape)
        target_affine = label_affine
        target_header = label.header
        target_dim_v = label.shape
        
#         labelmap2vol = npl.inv(target_affine).dot(vol_affine)
#         start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
#         sx, sy,sz = start_inv
#         end_inv = apply_affine(labelmap2vol, target_dim_v).astype(np.int32)
#         final_label = np.zeros(end_inv)
#         ex, ey, ez = end_inv
#         print("seg start inv v: ",start_inv , "segm end inv v:",end_inv)
        sx,sy,sz,ex,ey,ez = np.abs(get_points(vol, label))
        print(sx,sy,sz,ex,ey,ez)
        final_label = np.flip(final_label, axis=0)
#         final_label = np.flip(final_label, axis=1)
        final_label = np.flip(final_label, axis=2)
        final_label[0:ex+sx, 0:ey+sy, 0:ez+sz] = vol.get_fdata()
        
        final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
        volume, label = final_label_img, label
    
    return volume, label

In [ ]:
print("STARTING KORA LABEL-MAPS.")
print('Reading Label Maps.....')
for vol in file_paths.keys():
    print(vol)
    later = None
    print(file_paths[vol]['LABEL_PATHS'])
    if len(file_paths[vol]['LABEL_PATHS']) == 0:
        print(f"#################### ALERT:: NO LABELPATHS IN THE DICTIONARY FOR {vol} #########################")
        continue
        
    volume = nb.load(file_paths[vol]['N4_2']['OPP_CORRECTED'])
    img_ras_list = []
    for label_file_to_read in file_paths[vol]['LABEL_PATHS']:
        img_ras, lidx, labelname = read_ras(label_file_to_read, is_label=True)
        mode='constant'
        img_ras = resample_to_output(img_ras, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
    #     img_ras = resample_from_to(img_ras, [volume.shape, img_ras.affine])
        if labelname == 'SPLEEN':
            later = [img_ras, lidx, labelname]
        else:
            img_ras_list.append([img_ras, lidx, labelname])
    if later is not None:
        img_ras_list.append(later)
    
    s_label = label_parts(img_ras_list)
    volume, s_label = vol_label_fix(volume, s_label)
    #     volume, stitched_label = combine_nako_seg(file_paths[vol]['LABEL_PATHS'],'' ,file_paths[vol]['N4_2']['OPP_CORRECTED'])

    print('Viewing Stitched Images.....')
    volume_3_view_viewer(get_volume_data(volume))
    volume_3_view_viewer(get_volume_data(s_label))

    print('Saving Processed & Stitched Image.....')
    save_volume(volume, f'{processed_dir}/volume/{vol}')
    save_volume(s_label, f'{processed_dir}/label/{vol}')
    
print('FINISHED.')

In [ ]:
# file_dict= 
# {
#     VOL_ID: {
#         VOLUME_PATHS: {
#             OPP: [],
#             IN: [],
#             F: [],
#             W: []from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
#         },
#         LABEL_PATHS: [],
#         N4_1: [
#             {
#                 SCALED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled.nii.gz',
#                 IN_BIAS: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_bias_field.nii.gz',
#                 IN_CORRECTED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_corrected.nii.gz',
#                 OPP_CORRECTED: '',
#                 MIN: u_min,
#                 MAX: u_max
#             },
#             {},
#             {}
#         ],from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
#         ONE     : {
#             OPP: '',
#             IN: '',
#             F: '',
#             W: '',
#         },
#         N4_2: {
#             SCALED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled.nii.gz',
#             IN_BIAS: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_bias_field.nii.gz',
#             IN_CORRECTED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_corrected.nii.gz',
#             OPP_CORRECTED: '',
#             MIN: u_min,
#             MAX: u_max
#         },
        
#     }
# }